# A quick look at `ASTROQUERY`


In [1]:
import matplotlib.pyplot as plt
import numpy as np

from astropy.table import Table

import astropy.constants as c
import astropy.units as u

## `ASTROQUERY`

We'll load in a Simbad search from `astroquery` and use it to extract K-band magnitudes for several stars from the Simbad database.

In [2]:
from astroquery.simbad import Simbad

### The first step is to create a custom simbad search that allows us to add new VOTABLE codes to request.

In [4]:
# Create custom search
customSimbad = Simbad()

In [9]:
# List VOTABLE options
customSimbad.list_votable_fields()

--NOTES--

1. The parameter filtername must correspond to an existing filter. Filters include: B,V,R,I,J,K.  They are checked by SIMBAD but not astroquery.simbad

2. Fields beginning with rvz display the data as it is in the database. Fields beginning with rv force the display as a radial velocity. Fields beginning with z force the display as a redshift

3. For each measurement catalog, the VOTable contains all fields of the first measurement. When applicable, the first measurement is the mean one. 

Available VOTABLE fields:

bibcodelist(y1-y2)
biblio
cel
cl.g
coo(opt)
coo_bibcode
coo_err_angle
coo_err_maja
coo_err_mina
coo_qual
coo_wavelength
coordinates
dec(opt)
dec_prec
diameter
dim
dim_angle
dim_bibcode
dim_incl
dim_majaxis
dim_minaxis
dim_qual
dim_wavelength
dimensions
distance
distance_result
einstein
fe_h
flux(filtername)
flux_bibcode(filtername)
flux_error(filtername)
flux_name(filtername)
flux_qual(filtername)
flux_system(filtername)
flux_unit(filtername)
fluxdata(filtername)

### Now we activate the VOTABLE fields we want. We'll remove the coordinates column and enable K-band magnitudes.

In [8]:
customSimbad.remove_votable_fields('coordinates')
customSimbad.add_votable_fields('flux(K)','flux_error(K)','flux_system(K)')

/Users/howk/miniconda3/lib/python3.7/site-packages/astroquery/simbad/core.py:416: UserWarning: coordinates: this field is not set
  warnings.warn("{field}: this field is not set".format(field=field))


## Do a search

We will search on object names. First define the names in which we're interested, then do the search:

In [10]:
stars = ['HD233622','HD218915','HD14633']
simbaddata = customSimbad.query_objects(stars)

In [11]:
simbaddata.show_in_notebook()

idx,MAIN_ID,FLUX_K,FLUX_ERROR_K,FLUX_SYSTEM_K
,,mag,,
0,HD 233622,10.429,0.019,Vega
1,HD 218915,7.196,0.021,Vega
2,HD 14633,8.121,0.021,Vega


From here we can extract the magnitudes for our own use! Take care, as the data are in a masked table column; if we want to transform them to an array, the approach is a bit wonky...

In [51]:
Kmags = (simbaddata['FLUX_K']).data.data
Kmags

array([10.429,  7.196,  8.121], dtype=float32)

# Direct Gaia catalog search

Now we'll try a direct catalog search. In this case we need the coordinates, so we'll do a 2-step process: 

1. Get coordinates from SIMBAD
2. Get refined coordinates, magnitudes from Gaia

In [54]:
from astroquery.simbad import Simbad
from astroquery.gaia import Gaia

# Our objects
stars = ['HD233622','HD218915','HD14633']

# Create custom Simbad search
customSimbad = Simbad()
customSimbad.add_votable_fields('flux(K)','flux_error(K)','flux_system(K)')
simbaddata = customSimbad.query_objects(stars)
simbaddata.pprint()

 MAIN_ID        RA           DEC      ... FLUX_K FLUX_ERROR_K FLUX_SYSTEM_K
             "h:m:s"       "d:m:s"    ...  mag                             
--------- ------------- ------------- ... ------ ------------ -------------
HD 233622 09 21 33.6008 +50 05 56.366 ... 10.429        0.019          Vega
HD 218915 23 11 06.9480 +53 03 29.651 ...  7.196        0.021          Vega
HD  14633 02 22 54.2920 +41 28 47.723 ...  8.121        0.021          Vega


In [55]:
simbaddata.show_in_notebook()

idx,MAIN_ID,RA,DEC,RA_PREC,DEC_PREC,COO_ERR_MAJA,COO_ERR_MINA,COO_ERR_ANGLE,COO_QUAL,COO_WAVELENGTH,COO_BIBCODE,FLUX_K,FLUX_ERROR_K,FLUX_SYSTEM_K
,,"""h:m:s""","""d:m:s""",,,mas,mas,deg,,,,mag,,
0,HD 233622,09 21 33.6008,+50 05 56.366,14,14,0.048,0.046,90,A,O,2018yCat.1345....0G,10.429,0.019,Vega
1,HD 218915,23 11 06.9480,+53 03 29.651,14,14,0.035,0.034,90,A,O,2018yCat.1345....0G,7.196,0.021,Vega
2,HD 14633,02 22 54.2920,+41 28 47.723,14,14,0.118,0.077,90,A,O,2018yCat.1345....0G,8.121,0.021,Vega


#### Create the SkyCoord coordinates in order to do the Gaia search

In [44]:
from astropy.coordinates import SkyCoord

coords = SkyCoord(simbaddata['RA'],simbaddata['DEC'], unit=(u.hourangle, u.degree), frame='icrs')

#### Perform the Gaia search

It seems you need to search for one star at a time when using coordinates?

In [52]:
a=Gaia.cone_search_async(coords[2],radius=0.05*u.arcmin)

Query finished.


In [53]:
gg = a.get_data()
gg.show_in_notebook()

idx,dist,solution_id,designation,source_id,random_index,ref_epoch,ra,ra_error,dec,dec_error,parallax,parallax_error,parallax_over_error,pmra,pmra_error,pmdec,pmdec_error,ra_dec_corr,ra_parallax_corr,ra_pmra_corr,ra_pmdec_corr,dec_parallax_corr,dec_pmra_corr,dec_pmdec_corr,parallax_pmra_corr,parallax_pmdec_corr,pmra_pmdec_corr,astrometric_n_obs_al,astrometric_n_obs_ac,astrometric_n_good_obs_al,astrometric_n_bad_obs_al,astrometric_gof_al,astrometric_chi2_al,astrometric_excess_noise,astrometric_excess_noise_sig,astrometric_params_solved,astrometric_primary_flag,astrometric_weight_al,astrometric_pseudo_colour,astrometric_pseudo_colour_error,mean_varpi_factor_al,astrometric_matched_observations,visibility_periods_used,astrometric_sigma5d_max,frame_rotator_object_type,matched_observations,duplicated_source,phot_g_n_obs,phot_g_mean_flux,phot_g_mean_flux_error,phot_g_mean_flux_over_error,phot_g_mean_mag,phot_bp_n_obs,phot_bp_mean_flux,phot_bp_mean_flux_error,phot_bp_mean_flux_over_error,phot_bp_mean_mag,phot_rp_n_obs,phot_rp_mean_flux,phot_rp_mean_flux_error,phot_rp_mean_flux_over_error,phot_rp_mean_mag,phot_bp_rp_excess_factor,phot_proc_mode,bp_rp,bp_g,g_rp,radial_velocity,radial_velocity_error,rv_nb_transits,rv_template_teff,rv_template_logg,rv_template_fe_h,phot_variable_flag,l,b,ecl_lon,ecl_lat,priam_flags,teff_val,teff_percentile_lower,teff_percentile_upper,a_g_val,a_g_percentile_lower,a_g_percentile_upper,e_bp_min_rp_val,e_bp_min_rp_percentile_lower,e_bp_min_rp_percentile_upper,flame_flags,radius_val,radius_percentile_lower,radius_percentile_upper,lum_val,lum_percentile_lower,lum_percentile_upper,datalink_url,epoch_photometry_url
,,,,,,yr,deg,mas,deg,mas,mas,mas,,mas / yr,mas / yr,mas / yr,mas / yr,,,,,,,,,,,,,,,,,mas,,,,1 / mas2,1 / um,1 / um,,,,mas,,,,,electron / s,electron / s,,mag,,electron / s,electron / s,,mag,,electron / s,electron / s,,mag,,,mag,mag,mag,km / s,km / s,,K,log(cm.s**-2),dex,,deg,deg,deg,deg,,K,K,K,mag,mag,mag,mag,mag,mag,,Rsun,Rsun,Rsun,Lsun,Lsun,Lsun,,
0,3.068965180181487e-05,1635721458409799680,Gaia DR2 339061352757048576,339061352757048576,1420208327,2015.5,35.72621443474856,0.11835843642190819,41.47989300174014,0.07659247486657224,0.19805274950189342,0.12671823929313705,1.562938,-0.40277279244577624,0.18224723043314162,-6.996017967712323,0.16460217299960303,-0.19112848,0.67939997,-0.0055224868,0.13023035,-0.15630893,0.008108797,-0.021769358,0.033079922,0.18233244,-0.004052784,114,114,112,2,22.716328,899.28815,0.2715207578813032,19.699603977151476,31,True,10.916305,1.8623596040257107,0.026168900055632695,-0.13727054,13,9,0.09356257,0,18,False,139,20545585.35474411,24029.563207836596,855.0129,7.4065695,15,16617570.757283395,57031.84398929029,291.37354,7.299969,15,7131383.08083617,20522.195623912343,347.4961,7.6289854,1.1559151,1,-0.3290162,-0.106600285,-0.22241592,--,--,0,--,--,--,NOT_AVAILABLE,140.77786684484906,-18.20054148043417,47.54706018230171,25.701907417735143,100102,8868.667,7963.25,9593.0,--,--,--,--,--,--,--,--,--,--,--,--,--,http://geadata.esac.esa.int/data-server/datalink/links?ID=339061352757048576,--


In [58]:
gg['phot_g_mean_mag','phot_bp_mean_mag']

phot_g_mean_mag,phot_bp_mean_mag
mag,mag
float32,float32
7.4065695,7.299969
